In [1]:
import pandas as pd
import numpy as np
import cupy

def sigmoid(z):
    return cupy.reciprocal(1 + cupy.exp(-z))

def indicator(z):
    return 1 if z else 0

def indicator_len(z):
    return len(z) if len(z) != 0 else 1

In [37]:
U = cupy.array([
    [0.69254207,	0.120757424,	1],
    [0.794932131,	0.506465029,	1],
    [0.087887546,	0.625198524,	1],
    [0.186622288,	0.64901336,	1]
])

V = cupy.array([
    [0.961109006,	0.806823022,	0.906683013,	0.442439555],
    [0.613409475,	0.350356184,	0.527786582,	0.297789197],
    [0.451780345,	0.92009755,	0.848521427,	0.774361515]
])

In [31]:
est_pref = cupy.dot(U,V)

In [38]:
u = 0
I = {0}
#I = {}
#J = {2, 3}
J = {}
K = {1}
#K = {}

spec_alpha_u = 0.5
gamma = 0.01
lambda_u = 0.1
lambda_v = 0.1
lambda_b = 0.1

In [38]:
id(U[u, :-1])

2217152943880

In [39]:
U_u = U[u, :-1].copy()

In [40]:
r_hat_uI = cupy.average(est_pref[u, sorted(I)]) if len(
                    I) != 0 else cupy.array([0])
r_hat_uJ = cupy.average(est_pref[u, sorted(J)]) if len(
                    J) != 0 else cupy.array([0])
r_hat_uK = cupy.average(est_pref[u, sorted(K)]) if len(
                    K) != 0 else cupy.array([0])

r_hat_uIJ = r_hat_uI - r_hat_uJ
r_hat_uJK = r_hat_uJ - r_hat_uK

V_bar_I = cupy.average(V[:-1, sorted(I)], axis=1) if len(
    I) != 0 else cupy.zeros(
    shape=(2,))
V_bar_J = cupy.average(V[:-1, sorted(J)], axis=1) if len(
    J) != 0 else cupy.zeros(
    shape=(2,))
V_bar_K = cupy.average(V[:-1, sorted(K)], axis=1) if len(
    K) != 0 else cupy.zeros(
    shape=(2,))
# get b_I, b_J, b_K
b_I = cupy.average(V[-1, sorted(I)]) if len(I) != 0 else cupy.array([0])
b_J = cupy.average(V[-1, sorted(J)]) if len(J) != 0 else cupy.array([0])
b_K = cupy.average(V[-1, sorted(K)]) if len(K) != 0 else cupy.array([0])

In [41]:
indicator_I = indicator(len(I) == 0)
indicator_J = indicator(len(J) == 0)
indicator_K = indicator(len(K) == 0)
indicator_sum = indicator_I + indicator_J + indicator_K

In [42]:
if 0 <= indicator_sum <= 1:
    # these are the cases when only one set are empty or no set is empty
    # when all three are not empty, or I is empty, or K is empty, it is
    # easy to rewrite the obj by multiplying the indicator
    # when J is empty, we have to rewrite the obj
    if indicator_J == 1:
        # when J is empty
        r_hat_uIK = r_hat_uI - r_hat_uK

        # NABLA U_u
        df_dUu = sigmoid(- r_hat_uIK) * (V_bar_I - V_bar_K)
        dR_dUu = 2 * lambda_u * U_u
        # update U_u = U_u + gamma * (df_dUu - dR_dUu)
        U[u, :-1] += gamma * (df_dUu - dR_dUu)

        # NABLA V_i
        df_dbi = (1 - indicator_I) * sigmoid(- r_hat_uIK ) / indicator_len(I)
        dR_dbi = (1 - indicator_I) * 2 * lambda_b * b_I / indicator_len(I)
        df_dVi = df_dbi * U_u
        dR_dVi = 2 * lambda_v * V_bar_I / indicator_len(I)
        # update V_i = V_i + gamma * (df_dVi - dR_dVi)
        V[:-1, sorted(I)] += gamma * (df_dVi - dR_dVi)[:, None]  # trick: transpose here
        # update b_i = b_i + gamma * (df_dbi - dR_dbi)
        V[-1, sorted(I)] += gamma * (df_dbi - dR_dbi)

        # No change on J

        # NABLA V_k
        df_dbk = (1 - indicator_K) * - sigmoid(- r_hat_uIK) / indicator_len(K)
        dR_dbk = (1 - indicator_K) * 2 * lambda_b * b_K / indicator_len(K)
        df_dVk = df_dbk * U_u
        dR_dVk = 2 * lambda_v * V_bar_K / indicator_len(K)

        # update V_k = V_k + gamma * (df_dVk - dR_dVk)
        V[:-1, sorted(K)] += gamma * (df_dVk - dR_dVk)[:, None]  # trick: transpose here
        # update b_k = b_k + gamma * (df_dbk - dR_dbk)
        V[-1, sorted(K)] += gamma * (df_dbk - dR_dbk)

    else:
        # when J is not empty
        # NABLA U_u
        df_dUu = (1 - indicator_I) * sigmoid(- r_hat_uIJ / spec_alpha_u) / spec_alpha_u * (V_bar_I - V_bar_J) + \
 (1 - indicator_K) * sigmoid(- r_hat_uJK) * (V_bar_J - V_bar_K)
        dR_dUu = 2 * lambda_u * U_u
        # update U_u = U_u + gamma * (df_dUu - dR_dUu)
        U[u, :-1] += gamma * (df_dUu - dR_dUu)

        # NABLA V_i
        df_dbi = (1 - indicator_I) * sigmoid(- r_hat_uIJ / spec_alpha_u) / (indicator_len(I) * spec_alpha_u)
        dR_dbi = (1 - indicator_I) * 2 * lambda_b * b_I / indicator_len(I)
        df_dVi = df_dbi * U_u
        dR_dVi = 2 * lambda_v * V_bar_I / indicator_len(I)
        # update V_i = V_i + gamma * (df_dVi - dR_dVi)
        V[:-1, sorted(I)] += gamma * (df_dVi - dR_dVi)[:, None]  # trick: transpose here
        # update b_i = b_i + gamma * (df_dbi - dR_dbi)
        V[-1, sorted(I)] += gamma * (df_dbi - dR_dbi)

        # NABLA V_j
        df_dbj = (1 - indicator_I) * (- sigmoid(- r_hat_uIJ / spec_alpha_u) / spec_alpha_u +
          (1 - indicator_K) * sigmoid(- r_hat_uJK)) / indicator_len(J)
        dR_dbj = 2 * lambda_b * b_J / indicator_len(J)
        df_dVj = df_dbj * U_u
        dR_dVj = 2 * lambda_v * V_bar_J / indicator_len(J)

        # update V_j = V_j + gamma * (df_dVj - dR_dVj)
        V[:-1, sorted(J)] += gamma * (df_dVj - dR_dVj)[:, None]  # trick: transpose here
        # update b_j = b_j + gamma * (df_dbj - dR_dbj)
        V[-1, sorted(J)] += gamma * (df_dbj - dR_dbj)

        # NABLA V_k
        df_dbk = (1 - indicator_K) * - sigmoid(- r_hat_uJK) / indicator_len(K)
        dR_dbk = (1 - indicator_K) * 2 * lambda_b * b_K / indicator_len(K)
        df_dVk = df_dbk * U_u
        dR_dVk = 2 * lambda_v * V_bar_K / indicator_len(K)

        # update V_k = V_k + gamma * (df_dVk - dR_dVk)
        V[:-1, sorted(K)] += gamma * (df_dVk - dR_dVk)[:, None]  # trick: transpose here
        # update b_k = b_k + gamma * (df_dbk - dR_dbk)
        V[-1, sorted(K)] += gamma * (df_dbk - dR_dbk)

else:
    # these are the cases when at least two sets are empty
    # at these cases, we ignore this user and continue the loop
    print("ERROR CASE")

In [35]:
# new est pref
new_est_pref = cupy.dot(U,V)

In [46]:
for i in [0,1,2,3,4,5]:
    print(i)

0
1
2
3
4
5
